In [1]:
// # --- Day 11: Cosmic Expansion ---

// Puzzle description redacted as-per Advent of Code guidelines

// You may find the puzzle description at: https://adventofcode.com/2023/day/11

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2023, 11);
WriteLines(inputLines);

Loading puzzle file: Day11.txt
Total lines: 140
Max line length: 140

.....................#..........#......#............#........................................#..............................................
#.........................#...................................#...................#.........................................................
......................................................................#......#...................................#..........#...............
........................................................................................................................................#...
......#............................#.......................#................................................................................


In [4]:
record Point(int x, int y);

In [5]:
int Distance(Point a, Point b) {
    var across = a.x - b.x;
    across = across > 0 ? across : b.x - a.x;

    var vert = a.y - b.y;
    vert = vert > 0 ? vert : b.y - a.y;

    return across + vert;
}
var testDistance = Distance(new Point(1, 6), new Point(5, 11));
Console.WriteLine(testDistance);

9


In [6]:
IEnumerable<(int, int)> AllPairs(int range) {
    return Enumerable.Range(0, range).SelectMany(i => Enumerable.Range(i + 1, range - i - 1).Select(j => (i, j)));
}
foreach (var pair in AllPairs(5)) {
    Console.WriteLine(pair);
}

(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [7]:
List<Point> FindGalaxies(string[] universe) {
    var q = from y in Enumerable.Range(0, universe.Length)
            from x in Enumerable.Range(0, universe[y].Length)
            where universe[y][x] == '#'
            select new Point(x, y);
    return q.ToList();
}

In [8]:
const string testInput = """
....#........
.........#...
#............
.............
.............
........#....
.#...........
............#
.............
.............
.........#...
#....#.......
"""; // already expanded
var testInputLines = testInput.Split('\n');

foreach (var g in FindGalaxies(testInputLines)) {
    Console.WriteLine(g);
}

Point { x = 4, y = 0 }
Point { x = 9, y = 1 }
Point { x = 0, y = 2 }
Point { x = 8, y = 5 }
Point { x = 1, y = 6 }
Point { x = 12, y = 7 }
Point { x = 9, y = 10 }
Point { x = 0, y = 11 }
Point { x = 5, y = 11 }


In [9]:
int SumAllLengths(string[] inputLines) {
    var galaxies = FindGalaxies(inputLines);
    var galaxyPairs = AllPairs(galaxies.Count).Select((p) => (galaxies[p.Item1], galaxies[p.Item2]));

    var distanceSum = galaxyPairs.Select(pair => Distance(pair.Item1, pair.Item2)).Sum();

    return distanceSum;
}

// # In this example, after expanding the universe, the sum of the shortest path between all 36 pairs of galaxies is 374.
Console.WriteLine(SumAllLengths(testInputLines));

374


In [10]:
int[] FindEmptyRows(string[] inputLines) => Enumerable.Range(0, inputLines.Length).Where(row => inputLines[row].All(ch => ch == '.')).ToArray();
foreach (var row in FindEmptyRows(testInputLines)) {
    Console.WriteLine(row);
}

3
4
8
9


In [11]:
int[] FindEmptyCols(string[] inputLines) {
    var yRange = inputLines.Length;
    var xRange = inputLines[0].Length;

    return Enumerable.Range(0, xRange).Where(x => Enumerable.Range(0, yRange).All(y => inputLines[y][x] == '.')).ToArray();
}
foreach (var col in FindEmptyCols(testInputLines)) {
    Console.WriteLine(col);
}

2
3
6
7
10
11


In [12]:
IEnumerable<T> Repeat<T>(IEnumerable<T> source, int[] numsToRepeat) {
    var i = 0;
    foreach (var s in source) {
        yield return s;

        if (numsToRepeat.Contains(i)) {
            yield return s;
        }

        i++;
    }
}
Console.WriteLine(String.Concat(Repeat(".x.", [1])));

.xx.


In [13]:
const string testInputUnExpanded = """
...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
""";
var testInputUnExpandedLines = testInputUnExpanded.Split('\n');

In [14]:
int ExpandAndSumLengths(string[] inputLines) {
    var emptyRows = FindEmptyRows(inputLines);
    var emptyCols = FindEmptyCols(inputLines);

    var expandedUniverse = Repeat(inputLines, emptyRows).Select(row => String.Concat(Repeat(row, emptyCols))).ToArray();

    return SumAllLengths(expandedUniverse);
}
Console.WriteLine(ExpandAndSumLengths(testInputUnExpandedLines));

374


In [15]:
// # Expand the universe, then find the length of the shortest path between every pair of galaxies. What is the sum of these lengths?
var part1Answer = ExpandAndSumLengths(inputLines);
Console.WriteLine(part1Answer);

9734203


In [16]:
// 9734203 is correct!
Ensure(9734203, part1Answer);

In [17]:
// --- Part Two ---

// Puzzle description redacted as-per Advent of Code guidelines

// You may find the puzzle description at: https://adventofcode.com/2023/day/11

In [18]:
long ExpandedDistance(int expansion, Point a, Point b, int[]expandedRows, int[] expandedCols) {
    var xRange = a.x <= b.x ? (a.x, b.x) : (b.x, a.x);
    var yRange = a.y <= b.y ? (a.y, b.y) : (b.y, a.y);

    long across = xRange.Item2 - xRange.Item1;
    across += (expandedCols.Where(c => xRange.Item1 <= c && c <= xRange.Item2).Count() * expansion);

    long vert = yRange.Item2 - yRange.Item1;
    vert += (expandedRows.Where(r => yRange.Item1 <= r && r <= yRange.Item2).Count() * expansion);

    return across + vert;
}
Console.WriteLine(ExpandedDistance(1, new Point(0, 0), new Point(2, 2), [1], [1]));

6


In [19]:
long ExpandAndSumLengths2(int expansion, string[] inputLines) {
    var galaxies = FindGalaxies(inputLines);
    var galaxyPairs = AllPairs(galaxies.Count).Select((p) => (galaxies[p.Item1], galaxies[p.Item2]));

    var emptyRows = FindEmptyRows(inputLines);
    var emptyCols = FindEmptyCols(inputLines);
    var distanceSum = galaxyPairs.Select(pair => ExpandedDistance(expansion, pair.Item1, pair.Item2, emptyRows, emptyCols)).Sum();

    return distanceSum;
} 
// (In the example above, if each empty row or column were merely 10 times
// larger, the sum of the shortest paths between every pair of galaxies would be
// 1030. If each empty row or column were merely 100 times larger, the sum of the
// shortest paths between every pair of galaxies would be 8410. However, your
// universe will need to expand far beyond these values.)
Console.WriteLine(ExpandAndSumLengths2(9, testInputUnExpandedLines));


1030


In [ ]:
// Starting with the same initial image, expand the universe according to these
// new rules, then find the length of the shortest path between every pair of
// galaxies. What is the sum of these lengths?
int expansion = 1000000 - 1; // 
var part2Answer = ExpandAndSumLengths2(expansion, inputLines);
Console.WriteLine(part2Answer);

568914596391


In [21]:
// 568914596391 is correct!
Ensure(568914596391, part2Answer);